In [ ]:
import argilla as rg
from dotenv import load_dotenv
load_dotenv('D:\\gitFolders\\python_de_learners_data\\.env')
import os

rg.init(api_key="owner.apikey",
        api_url="https://kamaljp-argillatest.hf.space",
        extra_headers={"Authorization": f"Bearer {os.environ['HUGGING_FACE_KEY']}"})

In [ ]:
# create workspace
hfgilla = 'hfgilla'
rg.Workspace.create(name=hfgilla)

In [ ]:
hfws = rg.Workspace.from_name(hfgilla)
for d in hfws.list():
    print(d.name)

In [ ]:
user = rg.User.create(
    username="annotator1",
    first_name="anno",
    last_name="tator",
    password="12345678",
    role="annotator",
    workspaces=["hfgilla"]
)

In [ ]:
for u in rg.User.list():
    print(u.username)
    print(u.id)
    print(u.from_id(u.id))

In [ ]:
hfws.users

In [ ]:
# simple feddback_task_ds
fbds = rg.FeedbackDataset(
    guidelines='A simple FBDS with no records',
    fields=[
        rg.TextField(name="prompt", title='user_prompt'),
        rg.TextField(name="output", title='gen_output', use_markdown=True),
    ],
    questions=[
        rg.TextQuestion(
            name="corrected_text",
            title='Used for correcting the output',
            required=True,
            use_markdown=True
        ),
    ]
)

In [ ]:
fbds.push_to_argilla(name='fbds', workspace=hfgilla)

In [ ]:
fbds_1 = rg.FeedbackDataset(
    guidelines='Another fb dataset for practice',
    fields= [
        rg.TextField(name='model1_prompt', title='model_1'),
        rg.TextField(name="modelout", title='modelout', use_markdown=True)
    ],
    questions=[
        rg.RatingQuestion(
            name="rating",
            title="Rate the quality of the response:",
            description="1 = very bad - 5= very good",
            required=True,
            values=[1,2,3,4,5]  
        )
    ]
)
fbds_1.push_to_argilla("fbds2", workspace=hfgilla)

In [ ]:
# make some records
recfb = rg.FeedbackRecord(
    fields={
        "prompt": "this is funny",
        "output": "do you think so?"
    },
    metadata={'source': 'mind'},
    external_id='rec1'
)

In [ ]:
fbds.add_records([recfb])

In [ ]:
fbds.records

In [ ]:
fa_fbds = rg.FeedbackDataset.from_argilla('fbds', workspace=hfgilla)

In [ ]:
fa_fbds.add_records([recfb])

In [ ]:
rating_question = rg.RatingQuestion(
    name="rating",
    title="Rate the quality of the response:",
    description="1 = very bad - 5= very good",
    required=True,
    values=[1,2,3,4,5]  
)

In [ ]:
rec_tc = rg.TextClassificationRecord(
    text='this is a very huge and panaromic rock',
    prediction=[("view", 0.8), ('stay', 0.2)],
    annotation="view or stay",
)

In [ ]:
fa_fbds.add_records([rec_tc]) # will error out, as its not FBrecord

In [ ]:
rec_tc1 = rg.TextClassificationRecord(
    text='Lets climb the everest for sake of fun',
    prediction=[("adventure", 0.7), ('fun', 0.2), ('boring',0.1), ('worthless', 0.0)],
    annotation="fun or dun"
)

In [ ]:
rec_tc_ml = rg.TextClassificationRecord(
    text='There is always a lot more to explore',
    prediction=[("excited", 0.8), ('annoyed', 0.2)],
    annotation=["work", "play"],
    multi_label=True
)

In [ ]:
rg.log(records=[rec_tc, rec_tc1],
       name='tcds',
       workspace=hfgilla)

In [ ]:
rg.log(records=rec_tc1, name='tcds', workspace=hfgilla)

# more than one record can be pushed into dataset, if having same format

In [ ]:
# rg.log(records=rec_tc_ml, name='tcds', workspace=hfgilla)  # will throw error
rg.log(records=rec_tc_ml, name='tcml', workspace=hfgilla)

In [ ]:
tokrec = rg.TokenClassificationRecord(
    text="There is lot more tuna",
    tokens=["There", "is", "lot", "more", "tuna"],
    prediction=[("NOUN",0, 5), ("PRN", 6, 8), ("PLR", 9, 12)],
    annotation=[("NOUN", 0, 5)]
)

In [ ]:
rg.log(records=tokrec, name='tokds', workspace=hfgilla)

In [ ]:
t2tr = rg.Text2TextRecord(
    text="this is a very long record",
    prediction=["there is lot more than the record length"]
)
rg.log(records=t2tr, name='t2tr', workspace=hfgilla)

https://docs.argilla.io/en/v1.26.0/practical_guides/create_update_dataset/create_dataset.html

The records classes covered in this section correspond to three datasets: DatasetForTextClassification, DatasetForTokenClassification, and DatasetForText2Text. 

These will be deprecated in Argilla 2.0 and replaced by the fully configurable FeedbackDataset class. Not sure which dataset to use? Check out our section on choosing a dataset.

In [ ]:
import argilla as rg

fledgeds = rg.FeedbackDataset(
    guidelines="A fully fledged annotation pipeline",
    fields=[
        rg.TextField(name="input", title="Human input"),
        rg.TextField(name="output", title="Expected output", use_markdown=True)
    ],
    questions =[
        rg.RatingQuestion(
            name="rating",
            title="Rate the quality of the Output:",
            description="1 = very bad - 5= very good",
            required=True,
            values=[1,2,3,4,5]
        ),
        rg.TextQuestion(
            name="corrected-text",
            title="Provide a correction to the response:",
            required=False,
            use_markdown=True
        ),
        rg.LabelQuestion(
            name="relevant",
            title="Is the response relevant for the given prompt?",
            labels={"YES": "Yes", "NO": "No"}, # or ["YES","NO"]
            required=True,
            visible_labels=None
        ),
        rg.MultiLabelQuestion(
            name="content_class",
            title="Does the response include any of the following?",
            description="Select all that apply",
            labels={"hate": "Hate Speech" , "sexual": "Sexual content", "violent": "Violent content", "pii": "Personal information", "untruthful": "Untruthful info", "not_english": "Not English", "inappropriate": "Inappropriate content"}, # or ["hate", "sexual", "violent", "pii", "untruthful", "not_english", "inappropriate"]
            required=False,
            visible_labels=4
        ),
#         rg.SpanQuestion(
            # name="entities",
            # title="Highlight the entities in the text:",
            # labels={"PER": "Person", "ORG": "Organization", "EVE": "Event"},
            # # or ["PER", "ORG", "EVE"],
            # field="text",
            # required=True
        # ),
        rg.RankingQuestion(
            name="preference",
            title="Order replies based on your preference",
            description="1 = best, 3 = worst. Ties are allowed.",
            required=True,
            values={"reply-1": "Reply 1", "reply-2": "Reply 2", "reply-3": "Reply 3"} # or ["reply-1", "reply-2", "reply-3"]
        )
    ]
)

fledgeds.push_to_argilla(name="explds", workspace=hfgilla)

In [ ]:
# make some records
recfb = rg.FeedbackRecord(
    fields={
        "input": "this is funny",
        "output": "do you think so?"
    },
    metadata={'source': 'mind'},
    external_id='rec1'
)

In [ ]:
explds_argilla = rg.FeedbackDataset.from_argilla('explds', workspace=hfgilla)

In [ ]:
explds_argilla.add_records([recfb])

There is a new generation of datasets which are based on the task templates, and work well with hugging face datasets also. 

https://docs.argilla.io/en/v1.26.0/practical_guides/create_update_dataset/create_dataset.html

https://huggingface.co/datasets?other=argilla

These are easily integrated with HF, and pulled using below method.


In [ ]:
from datasets import load_dataset

dibt_dataset = load_dataset("DIBT/10k_prompts_ranked")

In [ ]:
dbt_ds = rg.FeedbackDataset.from_huggingface("nataliaElv/dolly_tutorial")

In [ ]:
dbt_ds[0:5]

In [ ]:
for user in rg.User.list():
    print(user.id)
    print(user.username)

In [ ]:
dbt_ds.push_to_argilla(name="dolly_hf", workspace=hfgilla,)

In [ ]:
# These templates include the fields and questions needed for the task,
# as well as the guidelines to provide to the annotators.

# After having initialized the FeedbackDataset templates, we can still alter 
# the fields, questions, guidelines, metadata and vectors to fit our specific
# needs you can refer to the update configuration section

In [ ]:
from argilla import FeedbackDataset, FeedbackRecord

In [ ]:
# Text Classification

ds_text_class = FeedbackDataset.for_text_classification(
    labels=["super", "duper"],
    multi_label=False,
    use_markdown=True,
    guidelines=None,
    metadata_properties=None,
    vectors_settings=None
) 

# FeedbackDataset(
#     fields=[
#         TextField(name="text", use_markdown=True)
#     ],
#     questions=[
#         LabelQuestion(name="label", labels=["positive", "negative"])
#     ],
#     guidelines="<Guidelines for the task>",
#     metadata_properties="<Metadata Properties>",
#     vectors_settings="<Vectors Settings>",
# )

text_class_record = FeedbackRecord(
    fields={
        "text": "this is funny",
    },
    # metadata={'source': 'mind'},
    # external_id='rec1'
)

ds_text_class.add_records([text_class_record])

ds_text_class.push_to_argilla(name='tc_template_rec',
                              workspace=hfgilla)


In [ ]:
ds_summ = FeedbackDataset.for_summarization(
    use_markdown=True,
    guidelines=None,
    metadata_properties=None,
    vectors_settings=None
)

text_summ_rec = FeedbackRecord(
    fields={
        "text": "this is a very long sentence and paragraphs of text"
    }
)
# FeedbackDataset(
#     fields=[
#         TextField(name="text", use_markdown=True)
#     ],
#     questions=[
#         TextQuestion(name="summary", use_markdown=True)
#     ],
#     guidelines="<Guidelines for the task>",
#     metadata_properties="<Metadata Properties>",
#     vectors_settings="<Vectors Settings>",
# )
ds_summ.add_records([text_summ_rec])
ds_summ.push_to_argilla("ds_summ", hfgilla)

In [ ]:
ds_trans = FeedbackDataset.for_translation(
    use_markdown=True,
    guidelines=None,
    metadata_properties=None,
    vectors_settings=None
)
text_trans = FeedbackRecord(
    fields={
        "source": "This is a foriegn language text to be translated."
    }
)
# FeedbackDataset(
#     fields=[
#         TextField(name="source", use_markdown=True)
#     ],
#     questions=[
#         TextQuestion(name="target", use_markdown=True)
#     ],
#     guidelines="<Guidelines for the task>",
#     metadata_properties="<Metadata Properties>",
#     vectors_settings="<Vectors Settings>",
# )

ds_trans.add_records([text_trans])
ds_trans.push_to_argilla("ds_trans", hfgilla)

In [ ]:
import argilla as rg

ds_t2t = rg.FeedbackDataset.for_natural_language_inference(
    labels=None,
    use_markdown=True,
    guidelines=None,
    metadata_properties=None,
    vectors_settings=None,
    )

t2t_rec = FeedbackRecord(
    fields={
        "premise":"This is the example premise",
        "hypothesis": "This is the example hypothesis"
    }
)

# FeedbackDataset(
#     fields=[
#         TextField(name="premise", use_markdown=True),
#         TextField(name="hypothesis", use_markdown=True)
#     ],
#     questions=[
#         LabelQuestion(name="label", labels=["entailment", "neutral", "contradiction"])
#     ],
#     guidelines="<Guidelines for the task>",
#     metadata_properties="<Metadata Properties>",
#     vectors_settings="<Vectors Settings>",
# )

ds_t2t.add_records([t2t_rec])
ds_t2t.push_to_argilla("ds_t2t", hfgilla)

In [ ]:
import argilla as rg

ds_sim = rg.FeedbackDataset.for_sentence_similarity(
    rating_scale=7,
    use_markdown=True,
    guidelines=None,
    metadata_properties=None,
    vectors_settings=None,
)

# FeedbackDataset(
#     fields=[
#         TextField(name="sentence-1", use_markdown=True),
#         TextField(name="sentence-2", use_markdown=True)
#     ],
#     questions=[
#         RatingQuestion(name="similarity", values=[1, 2, 3, 4, 5, 6, 7])
#     ],
#     guidelines="<Guidelines for the task>",
#     metadata_properties="<Metadata Properties>",
#     vectors_settings="<Vectors Settings>",
# )
sim_rec = FeedbackRecord(
    fields={
        "sentence-1":"This is the first sentence with a context",
        "sentence-2":"THis is second sentence similar context to first"
    }
)

ds_sim.add_records([sim_rec])
ds_sim.push_to_argilla("ds_sim", hfgilla)

In [ ]:
import argilla as rg

ds_qa = rg.FeedbackDataset.for_question_answering(
    use_markdown=True,
    guidelines=None,
    metadata_properties=None,
    vectors_settings=None,
)
# FeedbackDataset(
#     fields=[
#         TextField(name="question", use_markdown=True),
#         TextField(name="context", use_markdown=True)
#     ],
#     questions=[
#         TextQuestion(name="answer", use_markdown=True)
#     ],
#     guidelines="<Guidelines for the task>",
#     metadata_properties="<Metadata Properties>",
#     vectors_settings="<Vectors Settings>",
# )
qa_rec = FeedbackRecord(
    fields={
        "question": "This is where the question is",
        "context": "THis is the answer for above question"
    }
)

ds_qa.add_records([qa_rec])
ds_qa.push_to_argilla("ds_qa", hfgilla)




In [ ]:
import argilla as rg

ds_sft = rg.FeedbackDataset.for_supervised_fine_tuning(
    context=True,
    use_markdown=True,
    guidelines=None,
    metadata_properties=None,
    vectors_settings=None,
)
# FeedbackDataset(
#     fields=[
#         TextField(name="prompt", use_markdown=True),
#         TextField(name="context", use_markdown=True)
#     ],
#     questions=[
#         TextQuestion(name="response", use_markdown=True)
#     ],
#     guidelines="<Guidelines for the task>",
#     metadata_properties="<Metadata Properties>",
#     vectors_settings="<Vectors Settings>",
# )

sft_rec = FeedbackRecord(
    fields={
        "prompt": "This is the prompt from the user",
        "context": "Context provided to model during inference"
    }
)

ds_sft.add_records([sft_rec])
ds_sft.push_to_argilla('df_sft', hfgilla)


In [ ]:
import argilla as rg

ds_pref = rg.FeedbackDataset.for_preference_modeling(
    number_of_responses=2,
    context=False,
    use_markdown=True,
    guidelines=None,
    metadata_properties=None,
    vectors_settings=None,
)

# FeedbackDataset(
#     fields=[
#         TextField(name="prompt", use_markdown=True),
#         TextField(name="context", use_markdown=True),
#         TextField(name="response1", use_markdown=True),
#         TextField(name="response2", use_markdown=True),
#     ],
#     questions=[
#         RankingQuestion(name="preference", values=["Response 1", "Response 2"])
#     ],
#     guidelines="<Guidelines for the task>",
#     metadata_properties="<Metadata Properties>"
#     vectors_settings="<Vectors Settings>",
# )
pref_rec = FeedbackRecord(
    fields={
        "prompt": "This is the prompt from the user",
        "response1": "This is the response 1 from the model",
        "response2": "Response2 provided by the model during inference"
    }
)
ds_pref.add_records([pref_rec])
ds_pref.push_to_argilla("ds_pref", hfgilla)


In [ ]:
ds_pref.field_by_name('prompt')

In [ ]:
ds_pref.fields

In [ ]:
ds_pref.questions

In [ ]:
ds_pref.question_by_name("preference")

In [ ]:
import argilla as rg

ds_ppo = rg.FeedbackDataset.for_proximal_policy_optimization(
    rating_scale=7,
    context=True,
    use_markdown=True,
    guidelines=None,
    metadata_properties=None,
    vectors_settings=None,
)
# FeedbackDataset(
#     fields=[
#         TextField(name="prompt", use_markdown=True),
#         TextField(name="context", use_markdown=True)
#     ],
#     questions=[
#         TextQuestion(name="response", use_markdown=True)
#     ],
#     guidelines="<Guidelines for the task>",
#     metadata_properties="<Metadata Properties>",
#     vectors_settings="<Vectors Settings>",
# )
ppo_rec = FeedbackRecord(
    fields={
        "prompt":"This is the prompt to the model",
        "context": "This is the context given"
    }
)

ds_ppo.add_records(ppo_rec)
ds_ppo.push_to_argilla("ds_ppo", hfgilla)

In [ ]:
import argilla as rg

ds_dpo = rg.FeedbackDataset.for_direct_preference_optimization(
    number_of_responses=2,
    context=False,
    use_markdown=True,
    guidelines=None,
    metadata_properties=None,
    vectors_settings=None,
)
# FeedbackDataset(
#     fields=[
#         TextField(name="prompt", use_markdown=True),
#         TextField(name="context", use_markdown=True),
#         TextField(name="response1", use_markdown=True),
#         TextField(name="response2", use_markdown=True),
#     ],
#     questions=[
#         RankingQuestion(name="preference", values=["Response 1", "Response 2"])
#     ],
#     guidelines="<Guidelines for the task>",
#     metadata_properties="<Metadata Properties>",
#     vectors_settings="<Vectors Settings>",
# )
dpo_rec = FeedbackRecord(
    fields={
        "prompt":"This is the prompt",
        "context":"This is the context",
        "response1":"This is the response1",
        "response2":"This is the response2",
    }
)
ds_dpo.add_records([dpo_rec])
ds_dpo.push_to_argilla('ds_dpo', hfgilla)

In [ ]:
import argilla as rg

ds = rg.FeedbackDataset.for_retrieval_augmented_generation(
    number_of_retrievals=1,
    rating_scale=7,
    use_markdown=False,
    guidelines=None,
    metadata_properties=None,
    vectors_settings=None,
)
ds_rag = FeedbackRecord(
    fields={
        "query":"This is the question",
        "retrieved_document_1":"Retrieved document one",
    }
)
ds.add_records([ds_rag])
ds.push_to_argilla('ds_rag', hfgilla)
# FeedbackDataset(
#     fields=[
#         TextField(name="query", use_markdown=False),
#         TextField(name="retrieved_document_1", use_markdown=False),
#     ],
#     questions=[
#         RatingQuestion(name="rating_retrieved_document_1", values=[1, 2, 3, 4, 5, 6, 7]),
#         TextQuestion(name="response", use_markdown=False),
#     ],
#     guidelines="<Guidelines for the task>",
#     metadata_properties="<Metadata Properties>",
#     vectors_settings="<Vectors Settings>",
# )

In [ ]:
import argilla as rg

ds_mulclass = rg.FeedbackDataset.for_multi_modal_classification(
    labels=["video", "audio", "image"],
    multi_label=False,
    guidelines=None,
    metadata_properties=None,
    vectors_settings=None,
)
ds_rec = FeedbackRecord(
    fields={
        "content":"THis is the content in different modality"
    }
)
ds_mulclass.add_records([ds_rec])
ds_mulclass.push_to_argilla("ds_mulclass", hfgilla)
# FeedbackDataset(
#     fields=[
#         TextField(name="content", use_markdown=True, required=True),
#     ],
#     questions=[
#         LabelQuestion(name="label", labels=["video", "audio", "image"])
#     ],
#     guidelines="<Guidelines for the task>",
#     metadata_properties="<Metadata Properties>",
#     vectors_settings="<Vectors Settings>",
# )

In [ ]:
import argilla as rg

ds_trans = rg.FeedbackDataset.for_multi_modal_transcription(
    guidelines=None,
    metadata_properties=None,
    vectors_settings=None,
)
rec_trans = FeedbackRecord(
    fields={
        "content":"THis is a multi modal transcription"
    }
)
ds_trans.add_records([rec_trans])
ds_trans.push_to_argilla("ds_multrans", hfgilla)
# FeedbackDataset(
#     fields=[
#         TextField(name="content", use_markdown=True, required=True),
#     ],
#     questions=[
#         TextQuestion(name="description", use_markdown=True, required=True)
#     ],
#     guidelines="<Guidelines for the task>",
#     metadata_properties="<Metadata Properties>",
#     vectors_settings="<Vectors Settings>",
# )

**LabelQuestion:** These questions ask annotators to choose one label from a list of options. This type is useful for text classification tasks. In the UI, the labels of the LabelQuestion will have a rounded shape.

**MultiLabelQuestion:** These questions ask annotators to choose all applicable labels from a list of options. This type is useful for multi-label text classification tasks. In the UI, the labels of the MultiLabelQuestion will have a squared shape.

**RankingQuestion:** This question asks annotators to order a list of options. It is useful to gather information on the preference or relevance of a set of options. Ties are allowed and all options will need to be ranked.

**RatingQuestion:** These questions require annotators to select one option from a list of integer values. This type is useful for collecting numerical scores.

**SpanQuestion:** Here, annotators are asked to select a portion of the text of a specific field and apply a label to it. This type of question is useful for named entity recognition or information extraction tasks.

**TextQuestion:** These questions offer annotators a free-text area where they can enter any text. This type is useful for collecting natural language data, such as corrections or explanations

In [ ]:
from argilla import (
    LabelQuestion,
    MultiLabelQuestion,
    RankingQuestion,
    RatingQuestion,
    TextQuestion
)

lq = LabelQuestion(
    name="relevant",
    title="Is the response relevant for the given prompt?",
    labels={"YES": "Yes", "NO": "No"}, # or ["YES","NO"]
    required=True,
    visible_labels=None
)

mlq = MultiLabelQuestion(
    name="content_class",
    title="Does the response include any of the following?",
    description="Select all that apply",
    labels={"hate": "Hate Speech" , "sexual": "Sexual content", "violent": "Violent content", "pii": "Personal information", "untruthful": "Untruthful info", "not_english": "Not English", "inappropriate": "Inappropriate content"}, # or ["hate", "sexual", "violent", "pii", "untruthful", "not_english", "inappropriate"]
    required=False,
    visible_labels=4
)

rq = RankingQuestion(
    name="preference",
    title="Order replies based on your preference",
    description="1 = best, 3 = worst. Ties are allowed.",
    required=True,
    values={"reply-1": "Reply 1", "reply-2": "Reply 2", "reply-3": "Reply 3"} # or ["reply-1", "reply-2", "reply-3"]
)

raq = RatingQuestion(
    name="quality",
    title="Rate the quality of the response:",
    description="1 = very bad - 5= very good",
    required=True,
    values=[1, 2, 3, 4, 5]
)

tq = TextQuestion(
    name="corrected-text",
    title="Provide a correction to the response:",
    required=False,
    use_markdown=True
)

In [ ]:
from argilla import TextField
ds_qtest = FeedbackDataset(
    guidelines=None,
    fields=[
        TextField(name='prompt',required=True, use_markdown=True),
    ],
    metadata_properties=None,
    vectors_settings=None,    
    questions=[tq, raq, rq, mlq, lq]
)

In [ ]:
ds_qtest

In [ ]:
test_rek = FeedbackRecord(
    fields={
        "prompt":"This is for testing out"
    }
)
ds_qtest.add_records([test_rek])
ds_qtest.push_to_argilla("ds_qtest", hfgilla)

In [ ]:
# types of meta data
from argilla import (
    TermsMetadataProperty,
    IntegerMetadataProperty,
    FloatMetadataProperty
)

terms_md = TermsMetadataProperty(
    name="groups",
    title="Annotation groups",
    values=["group-a", "group-b", "group-c"] #optional
)

ds_qtest.add_metadata_property(terms_md)

int_md = IntegerMetadataProperty(
    name="integer-metadata",
    title="Integers",
    min=0, #optional
    max=100, #optional
    visible_for_annotators=False
)

ds_qtest.add_metadata_property(int_md)

fmd = FloatMetadataProperty(
    name="float-metadata",
    title="Floats",
    min=-0.45, #optional
    max=1000.34, #optional
    visible_for_annotators=False
)

ds_qtest.add_metadata_property(fmd)

In [ ]:
from argilla import VectorSettings
# You can use the add_vector_settings method. 
vectors_settings=[
    VectorSettings(
        name="sentence_embeddings",
        dimensions=768,
        title="Sentence Embeddings" #optional
    )
],

In [ ]:
dataset = rg.FeedbackDataset(
    fields=[
        rg.TextField(name="question"),
        rg.TextField(name="answer"),
    ],
    questions=[
        rg.RatingQuestion(
            name="answer_quality",
            description="How would you rate the quality of the answer?",
            values=[1, 2, 3, 4, 5],
        ),
        rg.TextQuestion(
            name="answer_correction",
            description="If you think the answer is not accurate, please, correct it.",
            required=False,
        ),
    ],
    metadata_properties = [
        rg.TermsMetadataProperty(
            name="groups",
            title="Annotation groups",
            values=["group-a", "group-b", "group-c"] #optional
        ),
        rg.FloatMetadataProperty(
            name="temperature",
            min=-0, #optional
            max=1, #optional
            visible_for_annotators=False
        )
    ],
    allow_extra_metadata = False,
    vectors_settings=[
        rg.VectorSettings(
            name="sentence_embeddings",
            dimensions=768,
            title="Sentence Embeddings" #optional
        )
    ],
    guidelines="Please, read the question carefully and try to answer it as accurately as possible."
)

In [ ]:
dataset.field_by_name("question")
# rg.TextField(name="question")
dataset.question_by_name("answer_quality")
# rg.RatingQuestion(
#     name="answer_quality",
#     description="How would you rate the quality of the answer?",
#     values=[1, 2, 3, 4, 5],
# )
dataset.metadata_property_by_name("groups")
# rg.TermsMetadataProperty(
#     name="groups",
#     title="Annotation groups",
#     values=["group-a", "group-b", "group-c"]
# )
# rg.VectorSettings(
#     name="sentence_embeddings",
#     title="Sentence Embeddings",
#     dimensions=768
# )
dataset.vector_settings_by_name("sentence_embeddings")

In [ ]:
remote_dataset = dataset.push_to_argilla(name="fulds", workspace=hfgilla)

In [ ]:
# updating details..
dataset = rg.FeedbackDataset(...)

new_fields=[
    TextField(),
    TextField(),
]

dataset.fields.extend(new_fields)
# Remove a non-required field
dataset.fields.pop(0)

# Add new questions
new_questions=[
    LabelQuestion(),
    MultiLabelQuestion(),
]

dataset.questions.extend(new_questions)

# Remove a non-required question
dataset.questions.pop(0)

# Add metadata properties
metadata = TermsMetadataProperty(name="my_metadata", values=["like", "dislike"])
dataset.add_metadata_property(metadata)

# Change metadata properties title
metadata_cfg = dataset.metadata_property_by_name("my_metadata")
metadata_cfg.title = "Likes"
dataset.update_metadata_properties(metadata_cfg)

# Delete a metadata property
dataset.delete_metadata_properties(metadata_properties="my_metadata")

# Add vector settings to the dataset
dataset.add_vector_settings(VectorSettings(name="my_vectors", dimensions=786))

# Change vector settings title
vector_cfg = ds.vector_settings_by_name("my_vectors")
vector_cfg.title = "Old vectors"
dataset.update_vectors_settings(vector_cfg)

# Delete vector settings
dataset.delete_vectors_settings("my_vectors")

# Define new guidelines for a question
dataset.questions[0].description = 'New description for the question.'